# Enquesta Condicions de vida dels usuaris dels CSS
# Tratamiento de datos
***
***
## 1. Preprocesado
***
### 1.1 Carga de archivos

In [4]:
import pandas as pd
import pyreadstat

# Cargar los archivos .sav
df_fitxa_llar, meta_fitxa_llar = pyreadstat.read_sav("data_raw/848_FITXA_LLAR.sav")
df_q_llar, meta_q_llar = pyreadstat.read_sav("data_raw/848_LLAR.sav")
df_individus, meta_individus = pyreadstat.read_sav("data_raw/848_INDIVIDUS.sav")

# Mostrar las primeras filas de cada DataFrame para inspección
with open("data_preprocess/1_variables_inicials.txt", "w") as f:

    f.write("848_FITXA_LLAR.sav\n")
    f.write(str(df_fitxa_llar.columns.tolist()))

    f.write("\n\n848_LLAR.sav\n")
    f.write(str(df_q_llar.columns.tolist()))

    f.write("\n\n848_INDIVIDUS.sav\n")
    f.write(str(df_individus.columns.tolist()))

print(f"Tamaño de df_fitxa_llar: {df_fitxa_llar.shape}")
print(f"Tamaño de df_q_llar: {df_q_llar.shape}")
print(f"Tamaño de df_individus: {df_individus.shape}")

Tamaño de df_fitxa_llar: (17809, 66)
Tamaño de df_q_llar: (6624, 419)
Tamaño de df_individus: (12288, 449)


Prueba para extraer información de los metadatos. Peta en un datetime, pero se puede explorar mejor en caso de necesitarlo

In [5]:
import json
import datetime

# Función personalizada para serializar objetos que no son compatibles con JSON, como datetime
def custom_serializer(obj):
    if isinstance(obj, (datetime.date, datetime.datetime)):
        return obj.isoformat()  # Convertir a formato ISO 8601
    raise TypeError(f"Tipo no serializable: {type(obj)}")

# Guardar los metadatos como JSON
with open("data_raw/fitxa_llar.json", "w") as file:
    json.dump(meta_fitxa_llar.__dict__, file, default=custom_serializer, indent=4)

with open("data_raw/q_llar.json", "w") as file:
    json.dump(meta_q_llar.__dict__, file, default=custom_serializer, indent=4)

with open("data_raw/individus.json", "w") as file:
    json.dump(meta_individus.__dict__, file, default=custom_serializer, indent=4)

***
### 1.2 Selección de variables

Se seleccionan las variables relevantes, eliminando aquellas que no interesan para el análisis. Se han limpiado a mano del primer campo del json con metadatos (`columns_names_to_labels`) y aquí se procesan para tener las versiones con las variables a utilizar finalmente.

In [6]:
import json

def list_variables(json_file: dict):
    return list(json_file['column_names_to_labels'].keys())

In [7]:
# Fitxa Llar
with open("data_preprocess/select_fitxa_llar.json", 'r', encoding='utf-8') as file:
    select_fitxa_llar = json.load(file)
variables_fitxa_llar = list_variables(select_fitxa_llar)
print(f"---- Fitxa Llar ----: \n{variables_fitxa_llar}\n")

# Questionari Llar
with open("data_preprocess/select_q_llar.json", 'r', encoding='utf-8') as file:
    select_q_llar = json.load(file)
variables_q_llar = list_variables(select_q_llar)
print(f"---- Questionari Llar ----: \n{variables_q_llar}\n")

# Individus
with open("data_preprocess/select_individus.json", 'r', encoding='utf-8') as file:
    select_individus = json.load(file)
variables_individus = list_variables(select_individus)
print(f"---- Individus ----: \n{variables_individus}\n")

---- Fitxa Llar ----: 
['IDENT_LL', 'IDENT', 'MEMBRE', 'DIST', 'BARRI', 'CSS', 'F9', 'F9PROV', 'F9PAIS1', 'F9PAIS2', 'F10', 'F13', 'F14', 'F13R', 'F11_12', 'INGRESSOS_EQUIVALENTS', 'TAXA60', 'TAXA40', 'TAXA30']

---- Questionari Llar ----: 
['IDENT_LL', 'IDENT', 'L20A1_1', 'L20A2_1', 'L20A1_2', 'L20A2_2', 'L20A1_3', 'L20A2_3', 'L21A', 'L21B', 'L22_3', 'L22_5', 'L24_1', 'L24_2', 'L48_1', 'L48_3', 'L49A', 'L49B', 'L49C', 'L49D', 'L53A1_1', 'L53A1_2', 'L53A1_3', 'L53A1_95', 'L53A1_98', 'L53A1_99', 'L53A2', 'L54', 'L48_2R', 'L23R', 'L20A1_8R', 'L20A1_6R', 'L20A1_4R', 'L20A1_5R', 'L22_1R', 'L22_2R', 'L22_4R', 'L22_6R', 'L22_7R', 'L50R', 'L52R', 'LP05_R1', 'LP06', 'LP01', 'L1_R1', 'HY010N', 'HY010G', 'HY020']

---- Individus ----: 
['IDENT_LL', 'IDENT', 'PERS_REF', 'P1', 'P2', 'P3', 'P35', 'P35_COD', 'P36H_1', 'P36M_1', 'P36H_2', 'P36M_2', 'P36H_3', 'P36M_3', 'P36H_4', 'P36M_4', 'P36H_5', 'P36M_5', 'P36H_6', 'P36M_6', 'P36H_7', 'P36M_7', 'P36H_8', 'P36M_8', 'P36H_9', 'P36M_9', 'P36H_10', 'P3

In [8]:
df_fitxa_llar = df_fitxa_llar.filter(variables_fitxa_llar, axis='columns')
df_q_llar = df_q_llar.filter(variables_q_llar, axis='columns')
df_individus = df_individus.filter(variables_individus, axis='columns')

#### Limpieza de metadatos JSON

Se genera un nuevo JSON que contiene unicamente los campos relativos a las variables seleccionadas.

In [9]:
import json

def clean_json(json_file: dict):
    variables = set(json_file['column_names_to_labels'].keys())
    if 'variable_value_labels' in json_file:
        json_file['variable_value_labels'] = {k: v for k, v in json_file['variable_value_labels'].items() if k in variables}
    if 'variable_measure' in json_file:
        json_file['variable_measure'] = {k: v for k, v in json_file['variable_measure'].items() if k in variables}
    if 'readstat_variable_types' in json_file:
        json_file['readstat_variable_types'] = {k: v for k, v in json_file['readstat_variable_types'].items() if k in variables}
    return json_file

In [10]:
# Fitxa Llar
with open("data_preprocess/select_fitxa_llar.json", 'r', encoding='utf-8') as file:
    select_fitxa_llar = json.load(file)
clean_fitxa_llar = clean_json(select_fitxa_llar)
with open('data_final/clean_fitxa_llar.json', 'w', encoding='utf-8') as file:
    json.dump(clean_fitxa_llar, file, ensure_ascii=False, indent=4)

# Questionari Llar
with open("data_preprocess/select_q_llar.json", 'r', encoding='utf-8') as file:
    select_q_llar = json.load(file)
clean_q_llar = clean_json(select_q_llar)
with open('data_final/clean_q_llar.json', 'w', encoding='utf-8') as file:
    json.dump(clean_q_llar, file, ensure_ascii=False, indent=4)

# Individus
with open("data_preprocess/select_individus.json", 'r', encoding='utf-8') as file:
    select_individus = json.load(file)
clean_individus = clean_json(select_individus)
with open('data_final/clean_individus.json', 'w', encoding='utf-8') as file:
    json.dump(clean_individus, file, ensure_ascii=False, indent=4)

In [11]:
# Mostrar las primeras filas de cada DataFrame para inspección
with open("data_preprocess/2_variables_truncades.txt", "w") as f:

    f.write("848_FITXA_LLAR.sav\n")
    f.write(str(df_fitxa_llar.columns.tolist()))

    f.write("\n\n848_LLAR.sav\n")
    f.write(str(df_q_llar.columns.tolist()))

    f.write("\n\n848_INDIVIDUS.sav\n")
    f.write(str(df_individus.columns.tolist()))

print(f"Tamaño de df_fitxa_llar: {df_fitxa_llar.shape}")
print(f"Tamaño de df_q_llar: {df_q_llar.shape}")
print(f"Tamaño de df_individus: {df_individus.shape}")

Tamaño de df_fitxa_llar: (17809, 19)
Tamaño de df_q_llar: (6624, 48)
Tamaño de df_individus: (12288, 109)


#### Procedimiento antiguo (eliminar variables con lista)

Para ir iterando en el proceso selección de variables se iban añadiendo a la lista aquellas que no eran relevantes.

In [12]:
# fitxa_llar_eliminar = ['ENTREVISTADOR', 'IDIOMA', 'COND_LEG_1', 'COND_LEG_2_1', 'COND_LEG_3_1', 'COND_LEG_4_1', 'COND_LEG_5_1', 'COND_LEG_5_1_1', 'COND_LEG_5_1_2', 'COND_LEG_5_1_3', 'COND_LEG_5_1_4', 'COND_LEG_5_1_5', 'COND_LEG_5_1_6', 
#                        'COND_LEG_6_1', 'PESAIXINDIV1', 'PESMOSINDIV1', 'PESAIXINDIV2', 'PESMOSINDIV2', 'PESAIXINDIV3', 'PESMOSINDIV3', 'F0A1', 'F0A2', 'F0', 'F2', 'F3', 'F4A1', 'F4A2', 'F5', 'F6', 'F7', 'F8A1', 'F8A2',
#                        'F11', 'F12', 'F15', 'F16A1_1', 'F16A1_2', 'F16A1_3', 'F16A1_4', 'F16A1_95', 'F16A1_96', 'F16A1_98', 'F16A1_99', 'F16A2', 'F8G', 'F8R1', 'F8R3']

In [13]:
# q_llar_eliminar = ['ENTREVISTADOR', 'REGISTRO', 'FECHAINI', 'HORAINI', 'FECHAFIN', 'HORAFIN', 'IDIOMA', 'COND_LEG_1', 'COND_LEG_2_1', 'COND_LEG_3_1', 'COND_LEG_4_1', 'COND_LEG_5_1', 'COND_LEG_5_1_1', 'COND_LEG_5_1_2', 'COND_LEG_5_1_3', 
#                     'COND_LEG_5_1_4', 'COND_LEG_5_1_5', 'COND_LEG_5_1_6', 'COND_LEG_6_1', 'PESLLAR_AIX1', 'PESLLAR_MOS1', 'PESLLAR_AIX2', 'PESLLAR_MOS2', 'PESLLAR_AIX3', 'PESLLAR_MOS3', 'PROXY', 'PROXI', 'L1TEXT', 
#                     'L2A1', 'L2A2', 'L3', 'L4A1', 'L4A2', 'L5', 'L6A1', 'L6A2', 'L7', 'L8', 'L9A1', 'L9A2', 'L10A1_1', 'L10A2_1', 'L10A2TEXT_1', 'L10A3_1', 'L10A3TEXT_1', 'L10A1_2', 'L10A2_2', 'L10A2TEXT_2', 'L10A3_2', 'L10A3TEXT_2', 'L10A1_3', 
#                     'L10A2_3', 'L10A2TEXT_3', 'L10A3_3', 'L10A3TEXT_3', 'L10A1_4', 'L10A2_4', 'L10A2TEXT_4', 'L10A3_4', 'L10A3TEXT_4', 'L10A1_5', 'L10A2_5', 'L10A2TEXT_5', 'L10A3_5', 'L10A3TEXT_5', 'L10A1_6', 'L10A2_6', 'L10A2TEXT_6', 'L10A3_6', 
#                     'L10A3TEXT_6', 'L10A1_7', 'L10A2_7', 'L10A2TEXT_7', 'L10A3_7', 'L10A3TEXT_7', 'L11A1', 'L11A2', 'L12A1', 'L12A2', 'L13A1', 'L13A2', 'L14A1_1', 'L14A2_1', 'L14A3_1', 'L14A3TEXT_1', 'L14A4_1', 'L14A4TEXT_1', 'L14A1_2', 'L14A2_2', 
#                     'L14A3_2', 'L14A3TEXT_2', 'L14A4_2', 'L14A4TEXT_2', 'L14A1_3', 'L14A2_3', 'L14A3_3', 'L14A3TEXT_3', 'L14A4_3', 'L14A4TEXT_3', 'L14A1_4', 'L14A2_4', 'L14A3_4', 'L14A3TEXT_4', 'L14A4_4', 'L14A4TEXT_4', 'L14A1_5', 'L14A2_5', 
#                     'L14A3_5', 'L14A3TEXT_5', 'L14A4_5', 'L14A4TEXT_5', 'L14A1_6', 'L14A2_6', 'L14A3_6', 'L14A3TEXT_6', 'L14A4_6', 'L14A4TEXT_6', 'L14A1_7', 'L14A2_7', 'L14A3_7', 'L14A3TEXT_7', 'L14A4_7', 'L14A4TEXT_7', 'L15A1', 'L15A2', 'L16A1', 
#                     'L16A2', 'L17A1_1', 'L17A2_1', 'L17A2TEXT_1', 'L17A3_1', 'L17A3TEXT_1', 'L17A1_2', 'L17A2_2', 'L17A2TEXT_2', 'L17A3_2', 'L17A3TEXT_2', 'L17A1_3', 'L17A2_3', 'L17A2TEXT_3', 'L17A3_3', 'L17A3TEXT_3', 'L17A1_4', 'L17A2_4', 'L17A2TEXT_4', 
#                     'L17A3_4', 'L17A3TEXT_4', 'L17A1_5', 'L17A2_5', 'L17A2TEXT_5', 'L17A3_5', 'L17A3TEXT_5', 'L17A1_6', 'L17A2_6', 'L17A2TEXT_6', 'L17A3_6', 'L17A3TEXT_6', 'L17A1_7', 'L17A2_7', 'L17A2TEXT_7', 'L17A3_7', 'L17A3TEXT_7', 'L18', 'L19A', 
#                     'L19ATEXT', 'L19B', 'L19BTEXT', 'L19C', 'L19CTEXT', 'L25', 'L26A1_1', 'L26A2_1', 'L26B_1', 'L26C_1', 'L26D1_1', 'L26D2_1', 'L26PRE', 'L26A1_2', 'L26A2_2', 'L26B_2', 'L26C_2', 'L26D1_2', 'L26D2_2', 'L26PRE2', 'L26A1_3', 'L26A2_3', 
#                     'L26B_3', 'L26C_3', 'L26D1_3', 'L26D2_3', 'L27', 'L28A1', 'L28A2', 'L29', 'L30', 'L31A1', 'L31A2', 'L32', 'L33A', 'L33ATEXT', 'L33B', 'L33BTEXT', 'L33C', 'L33CTEXT', 'L34', 'L35A', 'L35ATEXT', 'L35B', 'L35BTEXT', 'L35C', 'L35CTEXT', 
#                     'L36', 'L37A', 'L37ATEXT', 'L37B', 'L37BTEXT', 'L37C', 'L37CTEXT', 'L38', 'L39A', 'L39ATEXT', 'L39B', 'L39BTEXT', 'L39C', 'L39CTEXT', 'L40', 'L41A', 'L41ATEXT', 'L41B', 'L41BTEXT', 'L41C', 'L41CTEXT', 'L42', 'L43A_1', 'L43ATEXT_1', 
#                     'L43B_1', 'L43BTEXT_1', 'L43C_1', 'L43CTEXT_1', 'L43D_1', 'L43E_1', 'L43ETEXT_1', 'L43PRE', 'L43A_2', 'L43ATEXT_2', 'L43B_2', 'L43BTEXT_2', 'L43C_2', 'L43CTEXT_2', 'L43D_2', 'L43E_2', 'L43ETEXT_2', 'L43PRE2', 'L43A_3', 'L43ATEXT_3', 
#                     'L43B_3', 'L43BTEXT_3', 'L43C_3', 'L43CTEXT_3', 'L43D_3', 'L43E_3', 'L43ETEXT_3', 'L44', 'L45', 'L46', 'L46TEXT', 'L47', 'L51A1', 'L51A2', 'L55A', 'L55B', 'L55C', 'L55D', 'L55E', 'L55F', 'L55G', 'L55H', 'L55I','L56', 'L57A1', 
#                     'L57A2', 'OBS', 'LP04', 'LP05', 'LH01', 'LH01G', 'LH01R', 'LH01R2', 'LH02', 
#                     'HY100G', 'HY100N', 'HY030N', 'HY030G', 'HY040G', 'HY040N', 'HY060N', 'HY060G', 'PHY090G', 'PHY090N', 'HY070N', 'HY070G', 'HY080N', 'HY080G', 'HY110G', 'HY110N', 'HY131N', 'HY131G', 'HY130N', 'HY130G', 'HY120N', 'HY120G', 'PY010G', 
#                     'PY010N', 'PY021G', 'PY021N', 'PY020G', 'PY020N', 'PY035G', 'PY035N', 'PY050G', 'PY050N', 'PY080G', 'PY080N', 'PY090G', 'PY090N', 'PY100G', 'PY100N', 'PY110G', 'PY110N', 'PY120G', 'PY120N', 'PY130G', 'PY130N', 'PY140G', 'PY140N', 
#                     'PY150G', 'PY150N', 'PHY050G', 'PHY050N', 'HY145N', 'HY140', 'HY025', 'HY002', 'HY001',
#                     'DIST', 'BARRI', 'TAXA60', 'TAXA40', 'TAXA30']

In [14]:
# individus_eliminar = ['ESTUDIO', 'ENTREVISTADOR', 'REGISTRO', 'FECHAINI', 'HORAINI', 'FECHAFIN', 'HORAFIN', 'IDIOMA', 'PESAIXINDIV1', 'PESMOSINDIV1', 'PESAIXINDIV2', 'PESMOSINDIV2', 'PESAIXINDIV3', 'PESMOSINDIV3',
#                       'CONTESTA', 'MOTIU', 'MOTIU_COD', 'PROXY', 'PROXI', 'COND_LEG_1', 'COND_LEG_2_1', 'COND_LEG_3_1', 'COND_LEG_4_1', 'COND_LEG_5_1', 'COND_LEG_5_1_1', 'COND_LEG_5_1_2', 'COND_LEG_5_1_3', 'COND_LEG_5_1_4', 'COND_LEG_5_1_5', 
#                       'COND_LEG_5_1_6', 'COND_LEG_6_1', 'P1_COD', 'P2_O', 'P4_1', 'P4_2', 'P4_3', 'P4_98', 'P4_99', 'P4_1_COD', 'P4_2_COD', 'P4_3_COD', 'P5_0', 'P5_1', 'P5_2', 'P5_3', 'P6_1', 'P6_2', 'P7', 'P7_COD', 'P8_1', 'P8_2', 'P9', 'P10_1', 
#                       'P10_2', 'P10_3', 'P11', 'P12', 'P13_1', 'P13_2', 'P13_3', 'P13_4', 'P13_5', 'P14_1', 'P14_2', 'P14_3', 'P14_4', 'P14_5', 'P14_6', 'P15_1', 'P16_1', 'P16_2', 'P17_1', 'P17_2', 'P17_3', 'P18', 'P18_COD', 'P19', 'P19_COD', 'P20_1', 
#                       'P20_2', 'P20_3', 'P20_4', 'P20_98', 'P20_99', 'P20_COD', 'P21_1', 'P21_2', 'P22', 'P23_1', 'P23_2', 'P23_3', 'P24', 'P24_COD', 'P25', 'P26', 'P26_O', 'P27_1', 'P27_1_O', 'P27_2', 'P27_2_O', 'P27_3', 'P27_3_O', 'P27_4', 'P27_4_O', 
#                       'P27_5', 'P27_5_O', 'P27_6', 'P27_6_O', 'P27_7', 'P27_7_O', 'P27_8', 'P27_8_O', 'P27_9', 'P27_9_O', 'P27_10', 'P27_10_O', 'P27_11', 'P27_11_O', 'P27_12', 'P27_12_O', 'P28', 'P29', 'P29_O', 'P30_1', 'P30_1_O', 'P30_2', 'P30_2_O', 
#                       'P30_3', 'P30_3_O', 'P30_4', 'P30_4_O', 'P30_5', 'P30_5_O', 'P30_6', 'P30_6_O', 'P31', 'P32_1', 'P32_2', 'P33', 'P33_COD', 'P34', 'P34_COD', 'P37', 'P38', 'P39A_1', 'P39B_1', 'P39_1X', 'P39A_2', 'P39B_2', 'P39_2X', 'P39A_3', 'P39B_3', 
#                       'P39_3X', 'P40A', 'P40A_COD', 'P40B', 'P40C', 'P40D', 'P41_1', 'P41_2', 'P41_3', 'P41_4', 'P41_5', 'P41_6', 'P41_7', 'P41_8', 'P41_COD', 'P42', 'P43A_1', 'P43B_1', 'P43_1X', 'P43A_2', 'P43B_2', 'P43_2X', 'P43A_3', 'P43B_3', 'P43_3X', 
#                       'P44', 'P45', 'P46', 'P47', 'P48', 'P49', 'P50_1', 'P50A', 'P50_2', 'P50B', 'P50_98', 'P50_99', 'P51', 'P52', 'P53', 'P54_1', 'P54_2', 'P55', 'P56', 'P57', 'P58', 'P59', 'P60', 'P61', 'P62', 'P62_COD', 'P63', 'P64_1', 'P64_2', 'P64_3', 
#                       'P64_4', 'P65', 'P66', 'P67', 'P68', 'P69', 'P70', 'P71', 'P72', 'P73', 'P74', 'P76A_1', 'P76A_1_COD', 'P76B_1', 'P76B_1_O', 'P76C_1', 'P76D_1', 'P76E_1', 'P76F_1', 'P76_1X', 'P76A_2', 'P76A_2_COD', 'P76B_2', 'P76B_2_O', 'P76C_2', 
#                       'P76D_2', 'P76E_2', 'P76F_2', 'P76_2X', 'P76A_3', 'P76A_3_COD', 'P76B_3', 'P76B_3_O', 'P76C_3', 'P76D_3', 'P76E_3', 'P76F_3', 'P76_3X', 'P76A_4', 'P76A_4_COD', 'P76B_4', 'P76B_4_O', 'P76C_4', 'P76D_4', 'P76E_4', 'P76F_4', 'P76_4X', 
#                       'P76A_5', 'P76A_5_COD', 'P76B_5', 'P76B_5_O', 'P76C_5', 'P76D_5', 'P76E_5', 'P76F_5', 'P77', 'P78A_1', 'P78B_1', 'P78C_1', 'P78D_1', 'P78E_1', 'P78_X', 'P78A_2', 'P78B_2', 'P78C_2', 'P78D_2', 'P78E_2', 'P79', 'P80_1', 'P80_2', 'P81', 
#                       'P82', 'P83', 'P84_2', 'P14_1R', 'P14_2R', 'P6', 'P17', 'P23', 'LT012', 'PY010N', 'PY010G', 'PY021N', 'PY021G', 'PY020N', 'PY020G', 'PY035G', 'PY035N', 'PY050G', 'PY050N', 'PY080G', 'PY080N', 'PY090G', 'PY090N', 'PY100G', 'PY100N', 
#                       'PY110G', 'PY110N', 'PY120G', 'PY120N', 'PY130G', 'PY130N', 'PY140G', 'PY140N', 'PY150G', 'PY150N', 'PHY050G', 'PHY050N', 'TAXA60', 'TAXA40', 'TAXA30', 'L50R', 'L52R', 'F11_12', 'F13', 'F14', 'DIST', 'BARRI',
#                       # Estas son las recodificadas (que ya se añaden en q_llar!!):
#                       'L48_2R', 'L23R', 'L20A1_8R', 'L20A1_6R', 'L20A1_4R', 'L20A1_5R', 'L22_1R', 'L22_2R', 'L22_4R', 'L22_6R', 'L22_7R', 'LP01',
#                       # Otras:
#                       'MES', 'ANY']

In [15]:
# df_fitxa_llar = df_fitxa_llar.drop(fitxa_llar_eliminar, axis='columns')
# df_q_llar = df_q_llar.drop(q_llar_eliminar, axis='columns')
# df_individus = df_individus.drop(individus_eliminar, axis='columns')

***
### 1.3 Cambio a los valores reales de las variables

Se utilizan los campos de los metadatos (`variable_value_labels`) para cambiar los códigos de respuesta por su significado real. Así se convierten los números en strings para la mayoría de variables, que en realidad son categóricas.

In [16]:
with open("data_final/clean_fitxa_llar.json", 'r', encoding='utf-8') as file:
    metadata_fitxa_llar = json.load(file)
value_labels = metadata_fitxa_llar['variable_value_labels']
for var in value_labels.keys():
    if var in df_fitxa_llar.columns and var in value_labels:
        df_fitxa_llar[var] = df_fitxa_llar[var].map(lambda x: value_labels[var].get(str(x), x))

with open("data_final/clean_q_llar.json", 'r', encoding='utf-8') as file:
    metadata_q_llar = json.load(file)
value_labels = metadata_q_llar['variable_value_labels']
for var in value_labels.keys():
    if var in df_q_llar.columns and var in value_labels:
        df_q_llar[var] = df_q_llar[var].map(lambda x: value_labels[var].get(str(x), x))

with open("data_final/clean_individus.json", 'r', encoding='utf-8') as file:
    metadata_individus = json.load(file)
value_labels = metadata_individus['variable_value_labels']
for var in value_labels.keys():
    if var in df_individus.columns and var in value_labels:
        df_individus[var] = df_individus[var].map(lambda x: value_labels[var].get(str(x), x))

***
### 1.4 Unión de las tablas

In [17]:
# Primer paso: Unir df_fitxa_llar y df_q_llar usando 'IDENT_LL' y 'IDENT'
df_llar_merged = pd.merge(df_fitxa_llar, df_q_llar, on=['IDENT_LL', 'IDENT'], how='inner')

# Segundo paso: Unir df_llar_merged con df_individus usando 'IDENT_LL' y 'IDENT'
df = pd.merge(df_llar_merged, df_individus, on=['IDENT_LL', 'IDENT'], how='inner')

# Verificar el tamaño del DataFrame final
print("\n---- Tamaño del DataFrame combinado resultante ----")
print(df.shape)


---- Tamaño del DataFrame combinado resultante ----
(6624, 172)


#### Método para ver en que columnas compartidas entre tablas existen valores diferentes

In [18]:
# Paso 3: Identificar columnas duplicadas que han generado sufijos '_x' y '_y'
columnas_x_y = [col for col in df.columns if col.endswith('_x') or col.endswith('_y')]

# Paso 4: Comparar las columnas que tienen el mismo nombre pero con sufijos '_x' y '_y'
for col in columnas_x_y:
    col_base = col[:-2]  # Obtiene el nombre base de la columna sin el sufijo
    if col_base + '_x' in df.columns and col_base + '_y' in df.columns:
        # Compara si los valores de las columnas son iguales
        if df[col_base + '_x'].equals(df[col_base + '_y']):
            # Si son iguales, elimina una de las dos (mantiene solo '_x' o '_y')
            df = df.drop(columns=[col_base + '_y'])
            df = df.rename(columns={col_base + '_x': col_base})
        else:
            # Si no son iguales, puedes dejar ambas o inspeccionar los datos manualmente
            print(f'Valores diferentes en la columna: {col_base}')

In [19]:
with open("data_preprocess/3_variables_merged.txt", "w") as f:
    f.write("Variables del df resultante:\n")
    f.write(str(df.columns.tolist()))

***
### 1.5 Renombrar variables

En un json externo se ha realizado un mapeo del nombre original de las variables con el nuevo nombre para poder identificarlas de manera correcta

In [20]:
import json

with open('data_preprocess/map_variables.json', 'r') as json_file:
    column_mapping = json.load(json_file)

In [21]:
df = df.rename(columns=column_mapping)

In [22]:
with open("data_preprocess/4_variables_renombradas.txt", "w") as f:
    f.write("Variables del df resultante:\n")
    f.write(str(df.columns.tolist()))

In [23]:
df.to_pickle('data_preprocess/processed_df.pkl')
df.to_csv('data_preprocess/processed_df.csv', index=False)

***
### 1.6 Limpieza Valores Nulos

In [24]:
import pandas as pd

#df = pd.read_pickle('data_preprocess/processed_df.pkl')

In [23]:
# Análisis de valores nulos por variable
nulos_por_variable = df.isnull().sum()
porcentaje_nulos = (df.isnull().mean() * 100).round(2)

# Crear un DataFrame con el resumen de valores nulos
resumen_nulos = pd.DataFrame({
    "Variable": df.columns,
    "Nulos": nulos_por_variable,
    "Porcentaje Nulos": porcentaje_nulos
}).query("Nulos > 1000").reset_index(drop=True)

print("Resumen de valores nulos:")
print(resumen_nulos)

Resumen de valores nulos:
                     Variable  Nulos  Porcentaje Nulos
0             Prov_Nacimiento   2876             43.42
1            Pais1_Nacimiento   3748             56.58
2                     Conyuge   1708             25.79
3                N_Hijos_Llar   2852             43.06
4          Lavadora_Asequible   6257             94.46
5                TV_Asequible   6330             95.56
6          Telefono_Asequible   6576             99.28
7                 ONG_Caritas   2808             42.39
8               ONG_Creu_Roja   2808             42.39
9           ONG_Banc_Aliments   2808             42.39
10                  ONG_Otras   2808             42.39
11                     ONG_NS   2808             42.39
12                     ONG_NC   2808             42.39
13                 Tiempo_ONG   2808             42.39
14      Minutos_Limpieza_Ropa   1163             17.56
15    Minutos_Limpieza_Cocina   1159             17.50
16      Minutos_Limpieza_Casa   1214   

Se rellenan con un '0' las variables numéricas que tienen valor nulo cuando debería ser valor '0'.

In [24]:
df['N_Hijos_Llar'] = df['N_Hijos_Llar'].fillna(0)

# Nos deshacemos de las variables de Minutos
variables_minutos = ["Minutos_Limpieza_Ropa", "Minutos_Limpieza_Cocina", "Minutos_Limpieza_Casa", "Minutos_Cocinar", 
                     "Minutos_Comprar_Alim", "Minutos_Reparaciones", "Minutos_Admin_Dinero", "Minutos_Atencion_Ancianos", 
                     "Minutos_Atencion_Discapac", "Minutos_Atencion_Enfermos", "Minutos_Atencion_Menores"]
df = df.drop(variables_minutos, axis=1)

Eliminamos las variables que tienen más de un 90% de valores nulos: 'Lavadora_Asequible', 'TV_Asequible', 'Telefono_Asequible'. De hecho, las otras de asequible también tienen muchos valores nulos, y no aportan más valor que las de disponible...

In [25]:
df = df.drop(['Lavadora_Asequible', 'TV_Asequible', 'Telefono_Asequible'], axis=1)

#### Observaciones generales

- **31 casos**: Si `Propietat_Habitatge` == `Habitació col·lectiu`, tiene valores nulos en todas las respuestas de electrodomesticos
- **129 casos**: Valores nulos en estudios, situación laboralm, tareas domesticas y prestaciones.
- **408 casos**: Valores nulos en respuestas con feedback de serveis socials.

En la variable `Observaciones` de alguna de ellas, se comenta porque no está respondida la mayor parte de la entrevista. Estos casos habría que eliminarlos??

***
### 1.7 Creación nuevas variables

In [26]:
def generar_pais_nacimiento(row):
    if pd.notna(row['Prov_Nacimiento']):
        return "España"
    # elif row['Pais1_Nacimiento'] == "Altres":      #TODO: Revisar, los strings de Pais2_Nacimiento no son correctos.
    #     return row['Pais2_Nacimiento'].capitalize()
    else:
        return row['Pais1_Nacimiento']

# Crear la nueva columna
df['Pais_Nacimiento'] = df.apply(generar_pais_nacimiento, axis=1)

In [27]:
def determinar_tipo_hogar(conyuge, hijos):
    if conyuge == "Sí" and hijos == 0:
        return "Pareja sin hijos"
    elif conyuge == "Sí" and hijos > 0:
        return "Pareja con hijos"
    elif conyuge == "No" and hijos == 0:
        return "Persona sin hijos"
    elif conyuge == "No" and hijos > 0:
        return "Persona con hijos"
    else:
        return None  # Por si acaso hay valores inesperados

df['Tipo_Hogar'] = df.apply(lambda row: determinar_tipo_hogar(row['Conyuge_Recod'], row['N_Hijos_Llar']), axis=1)

In [28]:
nuevo_tipo_hogar_values = {
    'Unipersonal' : 'Unipersonal', 
    'Dues o més persones sense nucli' : 'Dos o más núcleos', 
    'Parella sense fills/es (sense altres persones)' : 'Pareja sin hijos',
    'Parella sense fills/es (amb altres persones)' : 'Pareja sin hijos',
    'Parella amb fills/es (sense altres persones)' : 'Pareja con hijos',
    'Parella amb fills/es (amb altres persones)' : 'Pareja con hijos',
    'Pare o mare amb fills/es (sense altres persones)' : 'Monoparental con hijos',
    'Pare o mare amb fills/es (amb altres persones)' : 'Monoparental con hijos',
    'Dos o més nuclis' : 'Dos o más núcleos'
}

# Recodificar y crear la nueva variable "Tipo_Hogar3"
df["Tipo_Hogar3"] = df["Tipo_Hogar2"].map(nuevo_tipo_hogar_values)

# Verificar el resultado (opcional)
print(df[["Tipo_Hogar2", "Tipo_Hogar3"]].head())

                                        Tipo_Hogar2             Tipo_Hogar3
0  Pare o mare amb fills/es (sense altres persones)  Monoparental con hijos
1      Parella amb fills/es (sense altres persones)        Pareja con hijos
2  Pare o mare amb fills/es (sense altres persones)  Monoparental con hijos
3                                       Unipersonal             Unipersonal
4                                       Unipersonal             Unipersonal


In [29]:
import numpy as np
import pandas as pd

# Definir los límites (bins) y las etiquetas
# Usamos 15 como límite inferior para que, al incluir el límite más bajo, se asignen correctamente a partir de 16.
bins = [0, 44, 64, 74, 84, np.inf]
etiquetas = ["16 - 44", "45 - 64", "65 - 74", "75 - 84", "85 i més"]

# Crear la nueva variable recodificada.
# Se asume que la columna 'Edad' existe y contiene valores numéricos.
df["Edad_Recod2"] = pd.cut(df["Edad"],
                          bins=bins,
                          labels=etiquetas,
                          right=True,           # Los límites superiores se incluyen (excepto el primer intervalo si no se usa include_lowest)
                          include_lowest=True)    # Asegura que el límite inferior se incluya en el primer bin

# Verifica el resultado:
print(df[["Edad", "Edad_Recod2"]].head())

   Edad Edad_Recod2
0  49.0     45 - 64
1  31.0     16 - 44
2  38.0     16 - 44
3  54.0     45 - 64
4  87.0    85 i més


Cambiamos algunos valores de las variables con horas de dedicación, para poder considerarlas como variables numéricas.

In [30]:
import pandas as pd

# Lista de variables de tiempo
variables_tiempo = ["Horas_Limpieza_Ropa", "Horas_Limpieza_Cocina",  "Horas_Limpieza_Casa", "Horas_Cocinar",
                    "Horas_Comprar_Alim", "Horas_Reparaciones", "Horas_Admin_Dinero"] 

for variable in variables_tiempo:
    # Convertir valores específicos a 0
    df[variable] = df[variable].replace("No s hi dedica", 0)
    # Asegurar que la columna sea numérica
    df[variable] = pd.to_numeric(df[variable], errors='coerce').fillna(0)

#"No en té la necessitat" !!Para el caso de Atencion a colectivos puede ser util....
variables_atencion = ["Horas_Atencion_Ancianos", "Horas_Atencion_Discapac", "Horas_Atencion_Enfermos", "Horas_Atencion_Menores"]

for variable in variables_atencion:
    # Convertir valores específicos a 0
    df[variable] = df[variable].replace("No en té la necessitat", 0)
    # Asegurar que la columna sea numérica
    df[variable] = pd.to_numeric(df[variable], errors='coerce').fillna(0)

In [31]:
# Lista de variables de tareas domésticas
domestic_tasks = [
    'Horas_Limpieza_Ropa', 
    'Horas_Limpieza_Cocina', 
    'Horas_Limpieza_Casa'
    # 'Horas_Cocinar', 
    # 'Horas_Comprar_Alim', 
    # 'Horas_Reparaciones', 
    # 'Horas_Admin_Dinero'
]

# Lista de variables de tareas de cuidados
care_tasks = [
    'Horas_Atencion_Ancianos', 
    'Horas_Atencion_Discapac', 
    'Horas_Atencion_Enfermos',  
    'Horas_Atencion_Menores'
]

# Crear la nueva variable 'Horas_Domesticas' sumando las columnas de tareas domésticas
df['Horas_Domesticas'] = df[domestic_tasks].sum(axis=1)

# Crear la nueva variable 'Horas_Cuidados' sumando las columnas de tareas de cuidados
df['Horas_Cuidados'] = df[care_tasks].sum(axis=1)

In [32]:
# Lista de variables de valoración
variables_valoracion = [
    "Valoracion_General", "Valoracion_TiempoEspera", "Valoracion_Facilidad",
    "Valoracion_Horarios", "Valoracion_Utilidad", "Valoracion_Dedicacion",
    "Valoracion_Intervencion", "Valoracion_Respuesta", "Valoracion_Profesionalidad",
    "Valoracion_Comprension", "Valoracion_Amabilidad", "Valoracion_Intimidad",
    "Valoracion_Utilidad2", "Valoracion_Gestion", "Valoracion_Amabilidad2",
    "Valoracion_Intimidad2", "Millora_Autonomia", "Millora_Anim",
    "Sentimient_Companyia", "Sentimient_Tranquilitat", "Sentimient_Seguretat",
    "Satisfaccio_Vital", "Benefici_Familia"
]

# Reemplazar valores específicos por códigos numéricos
valores_a_reemplazar = {"No ho sap": 98, "No contesta": 99}

for variable in variables_valoracion:
    # Reemplazar valores no numéricos por los códigos correspondientes
    df[variable] = df[variable].replace(valores_a_reemplazar)
    # Asegurar que la columna sea numérica
    df[variable] = pd.to_numeric(df[variable], errors='coerce').fillna(0)

Tener en cuenta que en estas variables de valoración numéricas aparecerán valores 98 y 99. Filtrar cuando se tengan que utilizar!! Se podría buscar un método para interpolar valores intermedios, pero no hay tantos valores nulos como para que valga la pena.

***
### 1.8 Eliminación variables no relevantes

Los paises han quedado capturados en la nueva variable 'Pais_Nacimiento'.

In [33]:
df = df.drop(['Pais1_Nacimiento', 'Pais2_Nacimiento'], axis=1)

'Membresía' y 'Parentesco' no tienen sentido al filtrar solo por la persona responsable de la llar. Igual para Persona de Referencia. Conyuge está mejor en la recodificada (se renombra a esa misma)

In [34]:
df = df.drop(['Membresia_Llar', 'Parentesco', 'Conyuge', 'Persona_Referencia'], axis=1)
df = df.rename(columns={"Conyuge_Recod": "Conyuge"})

Nos quedamos con Nivel de estudios recodificado

In [35]:
df = df.drop(['Nivel_Estudios'], axis=1)
df = df.rename(columns={"Nivel_Estudios_Recod": "Nivel_Estudios"})

También eliminamos los identificadores, que no tienen interés para el análisis.

In [36]:
df = df.drop(['ID_Hogar', 'ID_Individuo'], axis=1)

Por último, vamos a revisar valores nulos y acabar de eliminar algunas variables que no nos serán de utilidad

In [37]:
# Análisis de valores nulos por variable
nulos_por_variable = df.isnull().sum()
porcentaje_nulos = (df.isnull().mean() * 100).round(2)

# Crear un DataFrame con el resumen de valores nulos
resumen_nulos = pd.DataFrame({
    "Variable": df.columns,
    "Nulos": nulos_por_variable,
    "Porcentaje Nulos": porcentaje_nulos
}).query("Nulos > 500").reset_index(drop=True)

print("Resumen de valores nulos:")
print(resumen_nulos)

Resumen de valores nulos:
                 Variable  Nulos  Porcentaje Nulos
0         Prov_Nacimiento   2876             43.42
1             ONG_Caritas   2808             42.39
2           ONG_Creu_Roja   2808             42.39
3       ONG_Banc_Aliments   2808             42.39
4               ONG_Otras   2808             42.39
5                  ONG_NS   2808             42.39
6                  ONG_NC   2808             42.39
7              Tiempo_ONG   2808             42.39
8    Prestacio_Maternitat   5381             81.23
9     Prestacio_Jubilacio   5381             81.23
10         Prestacio_Atur   5381             81.23
11  Indemnizacion_Despido   5381             81.23


Las variables referentes a ONG concretas tienen demasiados valores nulos, y quedan recogidas en `Acudio_ONG` recodificada. Lo mismo para las referentes a prestaciones.

In [38]:
df = df.drop(['ONG_Caritas', 'ONG_Creu_Roja', 'ONG_Banc_Aliments', 'ONG_Otras', 'ONG_NS', 'ONG_NC', 'ONG_Otras2', 'Tiempo_ONG'], axis=1)
df = df.drop(['Prestacio_Maternitat', 'Prestacio_Jubilacio', 'Prestacio_Atur', 'Indemnizacion_Despido'], axis=1)

In [39]:
with open("data_final/5_variables_finales.txt", "w") as f:
    f.write("Variables del df resultante:\n")
    f.write(str(df.columns.tolist()))

***
### 1.9 Cambio de tipología de variables

In [40]:
# Diccionario de tipologias de variables
nuevas_tipologias = {
    "ID_Hogar": "float",
    "ID_Individuo": "float",
    # "Distrito": "category",
    # "Barrio": "category",
    # "Centro_CSS": "category",
    # "Lugar_Nacimiento": "category",
    # "Prov_Nacimiento": "category",
    # "Estado_Civil": "category",
    # "Conyuge": "category",
    "N_Hijos_Llar": "float",  # Puede ser int si garantizamos que no hay decimales
    "Ingresos_Equivalentes": "float",
    # "Riesgo_Pobreza_60": "category",
    # "Riesgo_Pobreza_40": "category",
    # "Riesgo_Pobreza_30": "category",
    # "Propietat_Habitatge": "category",
    # "Lavadora_Disponible": "category",
    # "Vater_Disponible": "category",
    # "Ducha_Disponible": "category",
    # "Goteras_Humedades": "category",
    # "Escasez_Luz_Natural": "category",
    # "Mala_Ventilacion": "category",
    # "Grietas_Paredes": "category",
    # "Problemas_Estructura": "category",
    # "Insectos_Ratas": "category",
    # "Ruido_Vecinos": "category",
    # "Temp_Caliente_Invierno": "category",
    # "Contaminacion_Medioambiental": "category",
    # "Delincuencia_Vandalismo": "category",
    # "Vacaciones_Anual": "category",
    # "Comida_DosDias": "category",
    # "Gasto_Imprevisto_800": "category",
    # "Mora_Hipoteca": "category",
    # "Mora_Prestamos": "category",
    # "Mora_Alquiler": "category",
    # "Mora_Servicios": "category",
    # "Fin_De_Mes": "category",
    # "Acudio_ONG": "category",
    # "Fin_De_Mes_Recod": "category",
    # "Acudio_ONG_Recod": "category",
    "Renda_Neta_Llar": "float",
    "Renda_Bruta_Llar": "float",
    "Renda_Total_Llar": "float",
    #"Persona_Referencia": "float",  # Asumiendo un identificador único
    # "Nivel_Estudios": "category", # Pasa a ser la recodificada
    # "Situacion_Laboral": "category",
    # "Categoria_Profesional": "category",
    # "Tareas_Domesticas": "category",
    # "Tareas_Domesticas_Otros": "category",
    "Horas_Limpieza_Ropa": "float",
    #"Minutos_Limpieza_Ropa": "float",
    "Horas_Limpieza_Cocina": "float",
    #"Minutos_Limpieza_Cocina": "float",
    "Horas_Limpieza_Casa": "float",
    #"Minutos_Limpieza_Casa": "float",
    "Horas_Cocinar": "float",
    #"Minutos_Cocinar": "float",
    "Horas_Comprar_Alim": "float",
    #"Minutos_Comprar_Alim": "float",
    "Horas_Reparaciones": "float",
    #"Minutos_Reparaciones": "float",
    "Horas_Admin_Dinero": "float",
    #"Minutos_Admin_Dinero": "float",
    "Horas_Atencion_Ancianos": "float",
    #"Minutos_Atencion_Ancianos": "float",
    "Horas_Atencion_Discapac": "float",
    #"Minutos_Atencion_Discapac": "float",
    "Horas_Atencion_Enfermos": "float",
    #"Minutos_Atencion_Enfermos": "float",
    "Horas_Atencion_Menores": "float",
    #"Minutos_Atencion_Menores": "float",
    # "Prestacion_Desempleo": "category",
    # "Prestacion_Jubilacion": "category",
    # "Prestacion_Supervivencia": "category",
    # "Prestacion_Proteccion": "category",
    # "Prestacion_Enfermedad": "category",
    # "Prestacion_Invalidez": "category",
    # "Prestacion_Escolar": "category",
    # "Prestacion_Otros": "category",
    # "Estado_Salud": "category",
    # "Nerviosismo": "category",
    # "Desanimo": "category",
    # "Tranquilidad": "category",
    # "Tristeza": "category",
    # "Felicidad": "category",
    # "Problemas_Salud": "category",
    # "Visita_CSS": "category",
    # "Visita_CSS_Econom": "category",
    # "Visita_CSS_Familia": "category",
    # "Visita_CSS_Depend": "category",
    # "Visita_CSS_Empleo": "category",
    # "Visita_CSS_Vivienda": "category",
    # "Visita_CSS_Otros": "category",
    # "Visita_CSS_Psico": "category",
    # "Visita_CSS_Salud": "category",
    # "Visita_CSS_Juridico": "category",
    # "Visita_CSS_Aliment": "category",
    # "Visita_CSS_ViolGenero": "category",
    # "Visita_CSS_NS": "category",
    # "Visita_CSS_NC": "category",
    # "Conoc_CSS_Boca": "category",
    # "Conoc_CSS_OAC": "category",
    # "Conoc_CSS_Web": "category",
    # "Conoc_CSS_Ayunt": "category",
    # "Conoc_CSS_ServPubl": "category",
    # "Conoc_CSS_ORG": "category",
    # "Conoc_CSS_Otros": "category",
    # "Conoc_CSS_Proxim": "category",
    # "Conoc_CSS_Iniciativa": "category",
    # "Conoc_CSS_NS": "category",
    # "Conoc_CSS_NC": "category",
    # "Conoc_CSS_Otros2": "category",
    "Valoracion_General": "float",
    "Valoracion_TiempoEspera": "float",
    "Valoracion_Facilidad": "float",
    "Valoracion_Horarios": "float",
    "Valoracion_Utilidad": "float",
    "Valoracion_Dedicacion": "float",
    "Valoracion_Intervencion": "float",
    "Valoracion_Respuesta": "float",
    "Valoracion_Profesionalidad": "float",
    "Valoracion_Comprension": "float",
    "Valoracion_Amabilidad": "float",
    "Valoracion_Intimidad": "float",
    "Valoracion_Utilidad2": "float",
    "Valoracion_Gestion": "float",
    "Valoracion_Amabilidad2": "float",
    "Valoracion_Intimidad2": "float",
    # "Recomend_CSS": "category",
    # "Participacion_Decision": "category",
    "Millora_Autonomia": "float",
    "Millora_Anim": "float",
    "Sentimient_Companyia": "float",
    "Sentimient_Tranquilitat": "float",
    "Sentimient_Seguretat": "float",
    "Satisfaccio_Vital": "float",
    # "Benefici_Familia": "category",
    # "Observaciones": "object",  # Texto libre
    # "Nivel_Estudios_Recod": "category",
    # "Estado_Salud_Recod": "category",
    # "Sexo": "category",
    "Edad": "float",
    "Membres_Llar": "float"
    # "Edad_Recod": "category",
    # "Segment_Gen_Edad": "category",
    # "Relacio_Activitat_Recod": "category",
    # "Pais_Nacimiento": "category",
    # "Tipo_Hogar": "category"
}

In [41]:
# Convertir los tipos de datos
for columna, tipo in nuevas_tipologias.items():
    try:
        # Comprobar si la columna es válida en el DataFrame
        if columna in df.columns:
            if tipo == "category":
                df[columna] = df[columna].astype("category")
            elif tipo == "int":
                df[columna] = pd.to_numeric(df[columna], errors="coerce").astype("Int64")
            elif tipo == "float":
                df[columna] = pd.to_numeric(df[columna], errors="coerce")
        else:
            print(f"Advertencia: La columna {columna} no existe en el DataFrame.")
    except Exception as e:
        print(f"Error al procesar la columna {columna}: {e}")

# Verificar los tipos de datos después de la conversión
print("Tipos de datos después de la conversión:")
print(df.dtypes)

Advertencia: La columna ID_Hogar no existe en el DataFrame.
Advertencia: La columna ID_Individuo no existe en el DataFrame.
Tipos de datos después de la conversión:
Distrito              object
Barrio                object
Centro_CSS            object
Lugar_Nacimiento      object
Prov_Nacimiento       object
                      ...   
Tipo_Hogar            object
Tipo_Hogar3           object
Edad_Recod2         category
Horas_Domesticas     float64
Horas_Cuidados       float64
Length: 143, dtype: object


Eliminamos Ingresos_Equivalentes, porque da errores la variable... Si es necesaria revisar más adelante

In [42]:
#df = df.drop(['Ingresos_Equivalentes'], axis=1)

***
### 1.10 Guardado de datos finales

In [43]:
import pandas as pd
import json

def export_variable_types_to_json(df, output_file):
    """
    Separa las variables del DataFrame en numéricas y categóricas,
    asignando a cada una un tipo simplificado (e.g. "int", "float", "categorical")
    y escribe esta información en un archivo JSON.
    
    Args:
        df (DataFrame): El DataFrame a analizar.
        output_file (str): Ruta y nombre del archivo JSON de salida.
    """
    numeric_types = {}
    categorical_types = {}
    
    for col, dtype in df.dtypes.items():
        # Para variables numéricas
        if pd.api.types.is_numeric_dtype(dtype):
            if pd.api.types.is_integer_dtype(dtype):
                numeric_types[col] = "int"
            elif pd.api.types.is_float_dtype(dtype):
                numeric_types[col] = "float"
            else:
                # Por si hubiera otro tipo numérico
                numeric_types[col] = str(dtype)
        # Para variables categóricas: consideramos tanto object como categorical
        elif pd.api.types.is_object_dtype(dtype) or isinstance(dtype, pd.CategoricalDtype):
            categorical_types[col] = "categorical"
        # Si quisieras incluir otros tipos, podrías agregarlos aquí.
    
    # Construir el diccionario final
    data = {
        "numericas": numeric_types,
        "categoricas": categorical_types
    }
    
    # Escribir en el archivo JSON con una indentación de 4 espacios para mayor legibilidad.
    with open(output_file, "w") as f:
        json.dump(data, f, indent=4)
    
    print(f"Archivo JSON guardado en: {output_file}")

export_variable_types_to_json(df, "data_final/variables_finales.json")

Archivo JSON guardado en: data_final/variables_finales.json


In [44]:
df.to_pickle('data_final/final_df.pkl')
df.to_csv('data_final/final_df.csv', index=False)